In [75]:
import pandas as pd

#import salling data that is a pickle file 
df = pd.read_pickle('data/salling_data.pkl')

#import recipe data
df2 = pd.read_csv('data/recipes_data.csv')

In [76]:
#keep the only columns we need
df = df[['Store Name', 'Category 1','Cleaned Ingredients', 'Original Price', 'New Price', 'Discount','Percent Discount']]
df.head()

,Store Name,Category 1,Cleaned Ingredients,Original Price,New Price,Discount,Percent Discount
1,Netto Lyngby,Hotdog Buns,hotdog bun,14.00,7.0,7.00,50.00
2,Netto Lyngby,Light Dark Buns,light dark bun,25.00,15.0,10.00,40.00
3,Netto Lyngby,Seeded Rye Breads,rye bread,7.75,6.0,1.75,22.58
4,Netto Lyngby,Buttermilk,buttermilk,13.25,10.0,3.25,24.53
5,Netto Lyngby,Other Pork,pork,59.95,36.0,23.95,39.95


In [83]:
df2.head()

,id,link,title,title_en,description_en,source
0,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,sunflower oil,bbc
1,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,onion,bbc
2,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,celery,bbc
3,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,carrots,bbc
4,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,split peas,bbc


In [84]:
#how many recipes we have, count unique titles
print(f"there are {df2['title'].nunique()} unique recipes")

#how many products we have, count unique description_en
print(f"there are {df2['description_en'].nunique()} unique ingredients")


there are 462 unique recipes
there are 603 unique ingredients


In [97]:
# Merge the dataframes on the 'Category 1' column from df and 'description_en' column from df2
merged_df = pd.merge(df2, df, left_on='description_en', right_on='Cleaned Ingredients', how='left')

# Display the merged dataframe
merged_df.head()

,id,link,title,title_en,description_en,source,Store Name,Category 1,Cleaned Ingredients,Original Price,New Price,Discount,Percent Discount
0,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,sunflower oil,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,onion,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,celery,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,carrots,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,https://www.bbc.co.uk/food/recipes/slow_cooker...,Slow cooker pea and ham soup,Slow cooker pea and ham soup,split peas,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
# Count unique recipes that have a match in the Salling data
print(f"There are {merged_df['title'].nunique()} unique recipes that have a match in the Salling data")

# Group by recipe title and count the unique matched ingredients
matched_counts = merged_df.dropna(subset=['Cleaned Ingredients']).groupby('title')['Category 1'].nunique().reset_index()
matched_counts.columns = ['title', 'matched_ingredients_count']

# Identify ingredients from the recipes that did not have a match
unmatched_ingredients = merged_df[merged_df['Cleaned Ingredients'].isna()]

# Count the number of unmatched ingredients for each recipe
unmatched_counts = unmatched_ingredients.groupby('title')['description_en'].nunique().reset_index()
unmatched_counts.columns = ['title', 'unmatched_ingredients_count']

# Merge the matched and unmatched counts into a single DataFrame
final_counts = pd.merge(matched_counts, unmatched_counts, on='title', how='outer').fillna(0)

# Display the final DataFrame
#final_counts['matched_ingredients'] = final_counts['title'].map(lambda x: merged_df[(merged_df['title'] == x) & (merged_df['Category 1'].notna())]['Cleaned Ingredients'].unique())
#final_counts['unmatched_ingredients'] = final_counts['title'].map(lambda x: merged_df[(merged_df['title'] == x) & (merged_df['Category 1'].isna())]['description_en'].unique())

final_counts.sort_values(by='matched_ingredients_count', ascending=False)

final_counts.head()

There are 462 unique recipes that have a match in the Salling data


,title,matched_ingredients_count,unmatched_ingredients_count
0,Apple crumble and caramel tart,1.0,13
1,Apricot upside-down cake,1.0,8
2,Beef Wellington,1.0,13
3,Beetroot and coconut soup,1.0,18
4,Beetroot stuffed paratha,1.0,16


In [116]:
merged_df[merged_df['title'] == 'Gluten-free creamy chicken and mash pie']

,id,link,title,title_en,description_en,source,Store Name,Category 1,Cleaned Ingredients,Original Price,New Price,Discount,Percent Discount
1933,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,potatoes,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1934,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,butter,bbc,føtex Lyngby Hovedgade,Other Butters,butter,22.95,14.0,8.95,39.00
1935,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,butter,bbc,Netto Gentoftegade,Other Butters,butter,16.95,4.0,12.95,76.40
1936,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,føtex Lyngby,Skimmed Milk,milk,17.95,9.0,8.95,49.86
1937,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,føtex Lyngby Hovedgade,Soured Milk,milk,17.50,8.0,9.50,54.29
1938,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,føtex Food Virum,Skimmed Milk,milk,19.00,15.0,4.00,21.05
1939,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,Netto Nærum,Semiskimmed Milk,milk,11.50,9.0,2.50,21.74
1940,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,Netto Gentoftegade,Semiskimmed Milk,milk,13.25,8.0,5.25,39.62
1941,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,wholegrain mustard,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1942,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,cheese,bbc,Netto Virum,Sliced Cheese,cheese,45.00,33.0,12.00,26.67


In [107]:
# display the recipe with more unique ingredients in merged_df
merged_df[merged_df['title'] == final_counts.loc[final_counts['matched_ingredients_count'].idxmax(), 'title']]

,id,link,title,title_en,description_en,source,Store Name,Category 1,Cleaned Ingredients,Original Price,New Price,Discount,Percent Discount
1933,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,potatoes,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1934,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,butter,bbc,føtex Lyngby Hovedgade,Other Butters,butter,22.95,14.0,8.95,39.00
1935,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,butter,bbc,Netto Gentoftegade,Other Butters,butter,16.95,4.0,12.95,76.40
1936,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,føtex Lyngby,Skimmed Milk,milk,17.95,9.0,8.95,49.86
1937,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,føtex Lyngby Hovedgade,Soured Milk,milk,17.50,8.0,9.50,54.29
1938,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,føtex Food Virum,Skimmed Milk,milk,19.00,15.0,4.00,21.05
1939,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,Netto Nærum,Semiskimmed Milk,milk,11.50,9.0,2.50,21.74
1940,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,milk,bbc,Netto Gentoftegade,Semiskimmed Milk,milk,13.25,8.0,5.25,39.62
1941,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,wholegrain mustard,bbc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1942,141,https://www.bbc.co.uk/food/recipes/gluten-free...,Gluten-free creamy chicken and mash pie,Gluten-free creamy chicken and mash pie,cheese,bbc,Netto Virum,Sliced Cheese,cheese,45.00,33.0,12.00,26.67
